# 按照自由時報那篇格式處理
[助教的程式碼](https://github.com/MiccWan/Political-News-Analysis/blob/master/crawler/new_talk_crawler.ipynb)

In [5]:
from selenium import webdriver
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from pathlib import Path, PureWindowsPath
from selenium.webdriver.chrome.options import Options

import sys
import pickle
import requests
from datetime import datetime


In [ ]:
def process_document(document, date):
    
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

In [7]:
res = requests.get('https://news.cnyes.com/news/cat/tw_stock_news')
doc = bs(res.text, 'lxml')
